In [1]:
import geopandas as gpd

In [2]:
gdf_quadras = gpd.read_file('data/SIRGAS_GPKG_quadraviariaed_2017.gpkg')

In [3]:
so_quadra = gdf_quadras.qe_tipo == 'Quadra'

In [4]:
gdf_quadras.shape

(64784, 6)

In [5]:
gdf_quadras = gdf_quadras[so_quadra].reset_index()

In [6]:
gdf_quadras.to_file('data/quadras_viarias.gpkg')

In [7]:
gdf_quadras.geometry = gdf_quadras.buffer(2)

In [8]:
gdf_quadras = gpd.read_file('data/quadras_viarias.gpkg')

In [9]:
# !gdalinfo data/2017/BHM-2017-1m.tiff

In [10]:
!gdal_rasterize -burn 1.0 -tr 1.0 1.0 -a_nodata 0.0 -te 313179.5 7343480.0 361220.5 7417015.0 -ot Float32 -of GTiff data/quadras_viarias.gpkg  data/quadras_viarias.tiff

0...10...20...30...40...50...60...70...80...90...100 - done.


In [12]:
# !gdal_calc.py -A data/2017/BHM-2017-1m.tiff -B data/2020/BHM-2020-1m.tiff -C data/quadras_viarias.tiff --outfile=data/delta/BHM-delta-2017-2020.tiff --calc="(B-A)*(B>A)*C" --NoDataValue=0 --hideNoData --overwrite

# !gdal_calc.py -A data/2017/BHM-2017-1m.tiff -B data/2020/BHM-2020-1m.tiff --outfile=data/delta/BHM-delta-2017-2020.tiff --calc="(B-A)*(B>A)" --NoDataValue=0 --overwrite

# !gdal_calc.py -A data/2017/BHM-2017-1m.tiff -B data/2020/BHM-2020-1m.tiff -C data/quadras_viarias.tiff --outfile=data/delta/BHM-delta-2017-2020.tiff --calc="(B-A)*C" --NoDataValue=0 --hideNoData --overwrite

!gdal_calc.py -A data/2017/MDS-sem-veg-2017-1m.tiff -B data/2020/MDS-sem-veg-2020-1m.tiff -C data/quadras_viarias.tiff --outfile=data/delta/MDS-sem-veg-acrescimo-2017-2020.tiff --calc="(B-A)*(B-A > 3.4)*C" --NoDataValue=0 --overwrite

!gdal_calc.py -A data/2017/MDS-sem-veg-2017-1m.tiff -B data/2020/MDS-sem-veg-2020-1m.tiff -C data/quadras_viarias.tiff --outfile=data/delta/MDS-sem-veg-decrescimo-2017-2020.tiff --calc="(B-A)*(B-A < -3.4)*C" --NoDataValue=0 --overwrite

14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 

In [5]:
# Executar o crivo/sieve
!gdal_sieve.py -st 10 -4 -nomask -of GTiff data/delta/MDS-sem-veg-acrescimo-2017-2020.tiff data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado.tiff

0...10...20...30...40...50...60...70...80...90...100 - done.


In [7]:
# Criar máscara para Edificacoes 2020
!gdal_fillnodata.py -md 5 -b 1 -of GTiff data/2020/BHM-2020-1m.tiff data/2020/BHM-2020-1m-5m-buffer.tiff

0...10...20...30...40...50...60...70..

In [ ]:
# Atribuindo todos os valores para 1
!gdal_calc.py -A data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado.tiff -B data/2020/BHM-2020-1m-5m-buffer.tiff --outflie=data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado-mascarado.tiff --calc="A*(B>0)" --NoDataValue=0 --overwrite

In [ ]:
# Preenche vazios
!gdal_fillnodata.py -md 1 -b 1 -of GTiff data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado-mascarado.tiff data/delta/MDS-sem-veg-acrescimo-2017-2020-crivado-mascarado-preenchido.tiff

In [4]:
# !gdal_sieve.py -st 10 -4 -nomask -of GTiff data/delta/MDS-sem-veg-decrescimo-2017-2020.tiff data/delta/MDS-sem-veg-decrescimo-2017-2020-crivado.tiff